# Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import csv
from google.colab import files 
from google.colab import drive
drive.mount('/content/drive')

directory = '/content/drive/My Drive/'

Mounted at /content/drive


# Обнаружение ошибок

In [2]:
column=['Index',	'Passenger_Id',	'Survived',	'Passenger_class',	'Full_Name',	'Sex',	'Age',	'SibSp',	'Parch',	'Ticket',	'Fare',	'Cabin',	'Embarked',	'familyName',	'firstName']

In [3]:
def check_error(table):
  '''
  Функция проверяет основные ошибки в массиве
  '''
  name_column=[]
  numder_meaning=[]
  error_empty_list=[]
  error_type_list=[]
  error_range_list=[]
  error_cross_field_list=[]
  error_unique_list=[]
  unique = []
  for i in column:
    for j in range(len(table)):
      name_column.append(i)
      numder_meaning.append(j)
      error_empty_list.append(bool(table[i][j]))
      error_type_list.append(type(table[i][j]))
      #error_unique_list.append("-")
      if column[2] in str(i): 
        if table[i][j] == 0 and type(table["Cabin"][j]) != str:
          error_cross_field_list.append("+")
        else:
          error_cross_field_list.append("-")
      else:
        error_cross_field_list.append("-")
      if column[2] in str(i): 
        if table[i][j] == 0 or table[i][j] ==1: 
          error_range_list.append("-")
        else:
          error_range_list.append(i)
      elif column[3] in str(i): 
        if table[i][j] < 0 and table[i][j] > 3: 
          error_range_list.append(i)
        else:
          error_range_list.append("-")  
      elif column[5] in str(i):
        if table[i][j]  == 'Male' or table[i][j] == 'Female': 
          error_range_list.append("-")
        else:
          error_range_list.append(i) 
      elif column[6] in str(i):
        if table[i][j] < 0 or table[i][j] > 100: 
          error_range_list.append(i)
        else:
          error_range_list.append("-")
      elif column[12] in str(i):
        if table[i][j] == "S" or table[i][j]  == "C" or table[i][j] == "Q": 
          error_range_list.append("-")
        else:
          error_range_list.append(i)
      elif column[14] in str(i):
        if table[i][j] == table['familyName'][j]: 
          error_range_list.append(i)
        else:
          error_range_list.append("-")
      elif column[9] in str(i):
        if len(table[i][j].split(' ')) == 2: 
          error_range_list.append(i)
        else:
          error_range_list.append("-")
      elif column[2] in str(i): 
        if table[i][j] == 0 or table[i][j] ==1: 
          error_range_list.append("-")
        else:
          error_range_list.append(i)
      elif column[1] in str(i): 
        if table[i][j] < 0 and table[i][j] > 891: 
          error_range_list.append(i)
        else:
          error_range_list.append("-")
      else:  
        error_range_list.append("-")    
      if column[1] in str(i): 
        if table[i][j] in unique:
          error_unique_list.append("+")
        else:
          unique.append(table[i][j]) 
          error_unique_list.append("-")   
      else:
        error_unique_list.append("-")   

  table_error=pd.DataFrame({'Имя_колонки':name_column,'Номер_строки':numder_meaning,'Пустые_строки':error_empty_list,'Тип_данных':error_type_list,'Ошибки_диапазонов':error_range_list,'Логические_ошибки':error_cross_field_list,'Ошибки_уникальности':error_unique_list})
  return table_error

# Исправление ошибок библиотек

In [4]:
data_survived = pd.read_csv(directory+'hw_prod13.csv')
data_survived.rename(columns={'Unnamed: 0': 'Index','PassengerId':'Passenger_Id','Pclass':'Passenger_class','Name':'Full_Name'}, inplace=True)

In [5]:
def counter(list_element):
    """Счетчик повторений элементов последовательности"""
    count = {}
    for element in list_element:
        if count.get(element, None):
            count[element] += 1
        else:
            count[element] = 1
    sorted_values = sorted(count.items(), key=lambda tpl: tpl[1], reverse=True)
    return dict(sorted_values)

In [6]:
def correction_data(table):
  '''
  Функция исправляет ошибки в соответствии с требованиями Data Quality Workflow
  '''
  unique3=[]
  count_error=[]
  new_table=table.copy()
  error_information=check_error(table)  
  new_table["Metric"] = "0"
  for i in range(len(error_information)):
    if "Age" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      new_table['Age'][index_row]=int(table['Age'].mean())
    if "Passenger_class" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      new_table['Passenger_class'][index_row]= min(table['Passenger_class'])
    if "Survived" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      new_table['Metric'][index_row]= "Missing"
    if "Passenger_Id" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      new_table['Metric'][index_row]= "Uncorrect"        
    if "Sex" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      if "M" in str(table['Sex'][index_row]) or "M." in str(table['Sex'][index_row]) or "male" in str(table['Sex'][index_row]) or "m" in str(table['Sex'][index_row]) or "m." in str(table['Sex'][index_row]): 
        new_table["Sex"][index_row]= 'Male' 
      elif "F" in str(table['Sex'][index_row]) or "F." in str(table['Sex'][index_row]) or "f" in str(table['Sex'][index_row]) or "f." in str(table['Sex'][index_row]) or "Frau" in str(table['Sex'][index_row]): 
        new_table["Sex"][index_row]= 'Female' 
      else:
        new_table['Metric'][index_row]= "Uncorrect"
    if "Ticket" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i]
      new_table['Ticket'][index_row]= table['Ticket'][index_row].split(' ')[1]
    if "Embarked" in str(error_information['Ошибки_диапазонов'][i]):
      index_row=error_information['Номер_строки'][i] 
      for g in range(len(table)):
        if table['Embarked'][g] in unique3:
          continue
        else:
          unique3.append(table['Embarked'][g]) 
      for uni in unique3:
        if type(uni) == str:
          continue
        else:
          unique3.remove(uni)
      for elem in unique3:   
        if len(table.loc[table['Embarked']==elem])*100/len(table['Embarked']) > 50:
          new_table['Embarked'][index_row]= elem 
    if error_information['Ошибки_диапазонов'][i] != "-":
      index_row=error_information['Номер_строки'][i]
      count_error.append(index_row)
      error_passanger=counter(count_error)
      if error_passanger[index_row] >= 4:
        new_table['Metric'][index_row]= "must_error_passanger"
    if "firstName" in str(error_information['Ошибки_диапазонов'][i]):
      if len(error_information.loc[error_information['Ошибки_диапазонов']=='firstName']) == len(table):
        clean_table=new_table.drop(columns='familyName')

    if "-" in str(error_information['Ошибки_уникальности'][i]):
      continue
    else:
      index_row=error_information['Номер_строки'][i]
      clean_table['Metric'][index_row]= "Dubble"

    if "-" in str(error_information['Логические_ошибки'][i]):
      continue
    else:
      index_row=error_information['Номер_строки'][i]
      clean_table['Metric'][index_row]= "Uncorrect"

    if "str" in str(error_information['Тип_данных'][i]):
      index_column=error_information['Имя_колонки'][i]
      index_row=error_information['Номер_строки'][i]
      if "Cabin" in str(index_column):
        try:
          clean_table[index_column][index_row]=table[index_column][index_row].split(' ')[0]
        except:
          clean_table[index_column][index_row]= "Missing"
      elif "Ticket" in str(index_column):
        try:
          if len(table[index_column][index_row].split(' ')[1]) == 1:
            continue
        except:
          clean_table[index_column][index_row]= "Missing" 
          continue         
    if "float" in str(error_information['Тип_данных'][i]):
      index_column=error_information['Имя_колонки'][i]
      index_row=error_information['Номер_строки'][i]
      try:
        clean_table[index_column][index_row]=int(table[index_column][index_row])
      except:
        clean_table[index_column][index_row]= "Missing"
        continue
        
  return clean_table

## Валидация исправлений

In [31]:
def varification(table):
  '''
  Функция проверяет корректность исправленных ошибок
  '''
  reestr_error_validation=[]
  for j in range(len(table)):
    if table['Passenger_Id'][j] > 0 or table['Passenger_Id'][j] < 892:
      continue
    else:
      reestr_error_validation.append(table['Passenger_Id'][j])
    if table['Survived'][j] == 0 or table['Survived'][j] ==1:
      continue
    else:
      reestr_error_validation.append(table['Survived'][j])
    if table['Passenger_class'][j] > 0 and table['Passenger_class'][j] < 4:
      continue
    else:
      reestr_error_validation.append(table['Passenger_class'][j])
    if table['Sex'][j]  == 'Male' or table['Sex'][j] == 'Female':
      continue
    else:
      reestr_error_validation.append(table['Sex'][j])
    if table['Embarked'][j]  == 'S' or table['Embarked'][j] == 'C' or table['Embarked'][j] == 'Q':
      continue
    else:
      reestr_error_validation.append(table['Embarked'][j])  
    if len(table['Ticket'][j].split(' ')) == 1:
      continue
    else:
      reestr_error_validation.append(table['Ticket'][j])
    for i in column:
      if type(table[i][j]) == mean(type(table[i])):
        continue
      else:
        reestr_error_validation.append("Ошибка типа данных: {}".format(table[i][j]))
  
  clean_table=table.loc[table['Metric'] == '0']
  if len(reestr_error_validation) ==0:
    data_survived_correct=clean_table.to_csv(directory+'data_survived_correct.csv')
    print(str("Варификация прошла успешно! Файл с исправлениями сохранен в пути: ")+str(directory)+str('data_survived_correct.csv'))
  else:
    print("Найдены ошибки верификации, см. реестр ошибок")

  return reestr_error_validation

In [32]:
result_verification=varification(correction_table)

Варификация прошла успешно! Файл с исправлениями сохранен в пути: /content/drive/My Drive/data_survived_correct.csv
